In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer,StandardScaler
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
df=pd.read_csv('Encoded_House.csv')

x=df.drop(['log_price','price'],axis=1).values
y=df['log_price'].values.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler_x = StandardScaler()
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)

X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [5]:
class HousePriceNN(nn.Module):
    def __init__(self, input_size):
        super(HousePriceNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [6]:
input_size = x_train.shape[1]
model = HousePriceNN(input_size)

In [7]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 100/1000, Loss: 0.3431
Epoch 200/1000, Loss: 0.2915
Epoch 300/1000, Loss: 0.2633
Epoch 400/1000, Loss: 0.2335
Epoch 500/1000, Loss: 0.2124
Epoch 600/1000, Loss: 0.1900
Epoch 700/1000, Loss: 0.1974
Epoch 800/1000, Loss: 0.1936
Epoch 900/1000, Loss: 0.1965
Epoch 1000/1000, Loss: 0.1809


In [9]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).numpy()
    true_values = y_test_tensor.numpy()

    print(f"\nNN Model [R²: {r2_score(true_values, predictions):.4f}, RMSE: {root_mean_squared_error(true_values, predictions):.4f}]")


NN Model [R²: 0.6262, RMSE: 0.4383]
